# Build Predictive Model for Smoke Detection

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

from sklearn.tree import DecisionTreeClassifier , plot_tree
from xgboost.xgbclassifier import XGBClassifier


ModuleNotFoundError: No module named 'xgboost'

### Data Preprocessing 

In [ ]:
df = pd.read_csv("smoke_detection_iot.csv",index_col=0)
df

In [ ]:
df.info()

In [ ]:
df.isna().sum()
#There is no null value in every attribute

In [ ]:
df.duplicated().sum()
#There is no duplicate data in this dataset

In [ ]:
df.describe()

In [ ]:
#Eliminate CNT and UTC, because it only shows counter of sample and timestamp respectively
df.drop(['UTC','CNT'],axis = 1,inplace=True)

Setelah dianalisa , diketahui bahwa tidak terdapat adanya null values maupun baris data yang terduplikat dari data ini, lalu setelah melakukan analisa dari data information yang ada, kami memutuskan untuk membuang dua buah kolom yang dirasa tidak begitu signifikan dalam modelling,yakni 'UTC' dan 'CNT'

### Exploratory Data Analysis

In [ ]:
#Analyse The Target Variable, which is fire alarm
df["Fire Alarm"].value_counts().plot(kind="bar")
plt.xticks([0, 1], ['No Alarm', 'Alarm'])
plt.xticks(rotation=0)

Terlihat bahwa kita sudah memiliki dataset yang baik, sebab data yang memperlihatkan bahwa fire alarm berbunyi lebih banyak dari yang tidak berbunyi, sehingga model kami dapat memprediksi apakah sebuah fire alarm berbunyi dengan baik

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 4, figsize = (15, 20))

for feature, subplot in zip(df, ax.flatten()):
    if feature == 'Fire Alarm':
        continue
    sns.boxplot(x = df['Fire Alarm'], y = df[feature], showfliers = False, ax = subplot)
    subplot.set_xticks([0, 1], ['No Alarm', 'Alarm'])

plt.suptitle('Feature Correlations with Target Variable (Fire Alarm)', fontsize = 20)
plt.tight_layout(pad = 2)
plt.show()

Terlihat bahwa:
1. Alarm yang berbunyi memiliki TVOC yang jauh lebih tinggi daripada alarm yang tidak berbunyi
2. Alarm yang berbunyi memiliki Raw Ethanol yang lebih rendah daripada alarm yang tidak berbunyi
3. Rata-rata Tekanan udara (Pressure) alarm yang tidak berbunyi lebih rendah daripada alarm yang berbunyi

In [ ]:
#Plotting correlation matrix heatmap

corr = df.corr()

plt.figure(figsize=(15,15))
sns.heatmap(corr,annot=True)

Terlihat bahwa terdapat multicollinnearity yang kuat pada PM1.0 , PM2.5 , NC0.5, NC1.0, dan NC 2.5. Untuk percobaan pertama, kami tidak ingin menghapus terlebih dahulu variabel yang multicollinear untuk melihat performa model secara raw.

# Feature engineering

In [ ]:
X = df.drop(['Fire Alarm'],axis=1)
Y = df['Fire Alarm']

regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]


Pada tahap ini, kami tidak melakukan feature scaling, sebab kami memerlukan nama-nama atribut yang ada untuk visualisasi decision tree plot, kemudian untuk nama kolom yang memiliki unsur [] atau < perlu diganti dengan '_', agar XGBClassifier dapat digunakan

In [ ]:
#Splitting train and test size with 20% test size
X_train,X_test,Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)


### DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier()

dt_model = dt.fit(X_train,Y_train)

dt_pred = dt.predict(X_test)

In [ ]:
print(classification_report(Y_test,dt_pred))

Terlihat bahwa model bekerja dengan sangat baik dengan akurasi 100%

In [ ]:
plt.figure(figsize=(35,20))
dtree_viz = plot_tree(dt_model, 
                      feature_names = df.drop(['Fire Alarm'],axis=1).columns,
                      class_names =['No Alarm','Alarm'])

plt.savefig('Model_dt.png')

Decision Tree plot could be shown on the .ipynb file , the plot shows that 30,136 from 35,825 rung alarm will ring in conditions : Pressure[hPa] > 938.143 hPa AND TVOC(ppb) > 204.5 ppb AND Humidity[%] >  46.21%  

The decision tree plot also shows that 9,060 from 14,279 non-rung alarm is under condition of : Pressure[hPa] <= 937.6 hPa  and the other 4,323 is under condition : Pressure[hPa] > 938.143 hPa AND TVOC[ppb] <= 87.5 ppb AND PM1.0 <= 1.455

In [ ]:
importance = dt_model.feature_importances_
importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': dt_model.feature_importances_
})
# summarize feature importance
#for i,v in enumerate(importance):
 #print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
#plt.bar([x for x in range(len(importance))], importance)
#plt.show()
plt.bar(x=importances['Attribute'], height=importances['Importance'], color='#087E8B')
plt.title('Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.show()

### XGBClassifier

In [ ]:
#Modelling dengan XGBoostClassifier
xgb = XGBClassifier()

xgb.fit(X_train,Y_train)

xgb_pred = xgb.predict(X_test)

In [ ]:
print(classification_report(Y_test,xgb_pred))
print(confusion_matrix(Y_test,xgb_pred))

Terlihat bahwa model bekerja dengan sangat sempurna dengan akurasi 100%(pembulatan, sebab terdapat 1 buah data yang salah).